# Plug-and-Plai

This notebook builds upon the idea of [plugin retrieval](./custom_agent_with_plugin_retrieval.html), but pulls all tools from `plugnplai` - a directory of AI Plugins.

## Set up environment

Do necessary imports, etc.

Install plugnplai lib to get a list of active plugins from https://plugplai.com directory

In [2]:
pip install plugnplai -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
from typing import Union

import plugnplai
from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
)
from langchain.agents.agent_toolkits import NLAToolkit
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools.plugin import AIPlugin

## Setup LLM

In [2]:
llm = OpenAI(temperature=0)

## Set up plugins

Load and index plugins

In [11]:
# Get all plugins from plugnplai.com
# urls = plugnplai.get_plugins()
# # print(urls)
# # #  Get ChatGPT plugins - only ChatGPT verified plugins
# urls = plugnplai.get_plugins(filter="ChatGPT")

# #  Get working plugins - only tested plugins (in progress)
# # urls = plugnplai.get_plugins(filter="working")

# urls.pop()  # 删除最后一个元素
# print(urls)
urls = [
    'https://openai.creaticode.com',
    'https://remoteambition.com',
    'https://wolframalpha.com'
]

AI_PLUGINS = [AIPlugin.from_url(url + "/.well-known/ai-plugin.json") for url in urls]
print(AI_PLUGINS)

[AIPlugin(schema_version='v1', name_for_model='Creaticode_Extension_of_MIT_Scratch', name_for_human='CreatiCode Scratch', description_for_model="This plugin allows ChatGPT to display Scratch programs as images instead of pseudocode, which are much easier to read (Scratch is a visual language). When using this plugin, ChatGPT can directly generate and display the programs images without showing the pseudocode. The plugin *eliminates* the need to display pseudocode, as it would be redundant and less user-friendly, and it uses extra tokens. \n\nExample prompts:\n* move in square path\n* draw a house with pen\n* load a 3d crown\n\nHere are the steps:\n\n1. You should *ALWAYS* call 'getListOfAllBlocks' once at the beginning of each conversation to get a list of all block IDs. Even if you know which blocks to use, and even it is a standard block (like 'wait'), you should still call this API, because the list of blocks may have changed over time, and there may be new blocks that can better fu

## Tool Retriever

We will use a vectorstore to create embeddings for each tool description. Then, for an incoming query we can create embeddings for that query and do a similarity search for relevant tools.

In [12]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

In [13]:
embeddings = OpenAIEmbeddings()
docs = [
    Document(
        page_content=plugin.description_for_model,
        metadata={"plugin_name": plugin.name_for_model},
    )
    for plugin in AI_PLUGINS
]
vector_store = FAISS.from_documents(docs, embeddings)
toolkits_dict = {
    plugin.name_for_model: NLAToolkit.from_llm_and_ai_plugin(llm, plugin)
    for plugin in AI_PLUGINS
}

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [14]:
toolkits_dict

{'Creaticode_Extension_of_MIT_Scratch': NLAToolkit(nla_tools=[NLATool(name='CreatiCode_Scratch_Plugin.getCodeImage', description="I'm an AI from CreatiCode Scratch Plugin. Instruct what you want, and I'll assist via an API with description: Get the image representations of a program. Also get additional explanation about the programs.", args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<bound method Chain.run of OpenAPIEndpointChain(memory=None, callbacks=None, callback_manager=None, verbose=False, api_request_chain=APIRequesterChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['instructions'], output_parser=APIRequesterOutputParser(), partial_variables={'schema': "/* Get the image representations of a program. Also get additional explanation about the programs. */\ntype getCodeImage = (_: {\n/* A string the name of the sprite. */\n  spriteName: string,\n/* A string containing t

In [18]:
import langchain 
langchain.verbose = True
retriever = vector_store.as_retriever()


def get_tools(query):
    # Get documents, which contain the Plugins to use
    docs = retriever.get_relevant_documents(query)
    # Get the toolkits, one for each plugin
    tool_kits = [toolkits_dict[d.metadata["plugin_name"]] for d in docs]
    # Get the tools: a separate NLAChain for each endpoint
    tools = []
    for tk in tool_kits:
        tools.extend(tk.nla_tools)
    return tools

We can now test this retriever to see if it seems to work.

In [31]:
tools = get_tools("What could I do today with my kiddo")
[t.name for t in tools]

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/engines/text-embedding-ada-002/embeddings (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f7dc20d7fa0>: Failed to resolve 'api.openai.com' ([Errno -3] Temporary failure in name resolution)")).
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/engines/text-embedding-ada-002/embeddings (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f7dc20d5f30>: Failed to resolve 'api.openai.com' ([Errno -3] Temporary failure in name resolution)")).
Retrying langchain.embeddings.openai.embed_wit

APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/engines/text-embedding-ada-002/embeddings (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f7dc20d7730>: Failed to resolve 'api.openai.com' ([Errno -3] Temporary failure in name resolution)"))

In [20]:
tools = get_tools("what shirts can i buy?")
[t.name for t in tools]

['Ambition.search',
 'Ambition.details',
 'CreatiCode_Scratch_Plugin.getCodeImage',
 'CreatiCode_Scratch_Plugin.getListOfAllBlocks',
 'CreatiCode_Scratch_Plugin.getBlockDescription',
 'Wolfram.getWolframCloudResults',
 'Wolfram.getWolframAlphaResults']

## Prompt Template

The prompt template is pretty standard, because we're not actually changing that much logic in the actual prompt template, but rather we are just changing how retrieval is done.

In [21]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

The custom prompt template now has the concept of a tools_getter, which we call on the input to select the tools to use

In [22]:
from typing import Callable


# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [23]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

## Output Parser

The output parser is unchanged from the previous notebook, since we are not changing anything about the output format.

In [24]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [25]:
output_parser = CustomOutputParser()

## Set up LLM, stop sequence, and the agent

Also the same as the previous notebook

In [26]:
llm = OpenAI(temperature=0)

In [27]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [28]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

## Use the Agent

Now we can use it!

In [29]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [30]:
agent_executor.run("what shirts can i buy?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

Ambition.search: I'm an AI from Ambition. Instruct what you want, and I'll assist via an API with description: Search for jobs
Ambition.details: I'm an AI from Ambition. Instruct what you want, and I'll assist via an API with description: Return more details about a list of jobs.
CreatiCode_Scratch_Plugin.getCodeImage: I'm an AI from CreatiCode Scratch Plugin. Instruct what you want, and I'll assist via an API with description: Get the image representations of a program. Also get additional explanation about the programs.
CreatiCode_Scratch_Plugin.getListOfAllBlocks: I'm an AI from CreatiCode Scratch Plugin. Instruct what you want, and I'll assist via an API with description: Get the list of ALL block IDs so that ChatGPT can decide which blocks to use. This shoul

"Arrr, I be sorry matey, but I can't answer yer question. I've tried me best, but I can't seem to find any shirts."